In [1]:
from pycaret.utils import version
version()

'3.3.2'

### reading and cleaning data

In [2]:
import pandas as pd

In [4]:
df_ind = pd.read_csv('../data_ind.csv')
df_oecd = pd.read_csv('../data_oecd.csv')

In [5]:
df_oecd = df_oecd.sort_values(by=['Country', 'Time_Period'])
df_oecd['Time_Period'] = pd.to_datetime(df_oecd['Time_Period'], format='%Y-%m')

df_deu = df_oecd[(df_oecd['Country'] == 'DEU')]
df_usa = df_oecd[(df_oecd['Country'] == 'USA')]

df_ind['Time_Period'] = df_ind['Year'].astype(str) + '-' + df_ind['Quarter'].map({'-': '01', 'Q1': '02', 'Q3': '08'})
df_ind['Country'] = 'IND'  # Add country column
df_ind = df_ind[['Country', 'Time_Period', 'Unemployment_Rate']]
df_ind['Time_Period'] = pd.to_datetime(df_ind['Time_Period'], format='%Y-%m')

In [6]:
select_df_ind = df_ind[df_ind['Time_Period'].dt.year >= 2005]
select_df_deu = df_deu[df_deu['Time_Period'].dt.year >= 2005]
select_df_usa = df_usa[df_usa['Time_Period'].dt.year >= 2005]
